In [20]:
import requests, json, us

In [2]:
data = json.loads(requests.get('https://projects.fivethirtyeight.com/2018-midterm-election-forecast/house/home.json').content)

In [5]:
data = data['districtForecasts']

In [34]:
def process_district(d):
    res = {}
    res['state'] = d['state']
    res['district'] = d['district']
    for c in d['forecast']:
        if c['party'] == 'D':
            res['demcandidate'] = c['candidate']
        elif c['party'] == 'R':
            res['repcandidate'] = c['candidate']
    res['dem_incumbent'] = False
    res['rep_incumbent'] = False
    for i in d['incumbents']:
        if i['party'] == "R":
            res['rep_incumbent'] = True
        elif i['party'] == "D":
            res['dem_incumbent'] = True
    
    return res

In [35]:
current = pd.DataFrame([process_district(d) for d in data]).query('state != "US"')

In [36]:
current['state_fips'] = current['state'].apply(lambda x : us.states.lookup(x).fips)

In [37]:
current = current.rename(columns={'district' : 'fivethirtyeight_district'})

In [40]:
def generate_district(x):
    if len(x) == 1:
        x['district'] = 0
    else:
        x['district'] = x['fivethirtyeight_district']
    return x

current = current.groupby('state_fips').apply(generate_district)

In [47]:
(current.reset_index(drop=True)
 .astype({'fivethirtyeight_district':'str', 'state_fips': 'str', 'district':int})
 .drop('state', axis=1)
 .to_feather('data/main_2018.feather'))